## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-06-02-27-01 +0000,bbc,Clintons subpoenaed to testify in congressiona...,https://www.bbc.com/news/articles/c79l38vl3lwo...
1,2025-08-06-02-21-41 +0000,bbc,RFK Jr cancels $500m in mRNA vaccine developme...,https://www.bbc.com/news/articles/c74dzdddvmjo...
2,2025-08-06-02-07-58 +0000,nypost,Oasis fan who plunged to his death during Lond...,https://nypost.com/2025/08/05/world-news/oasis...
3,2025-08-06-02-06-00 +0000,startribune,Elusive bear survives more than a week with a ...,https://www.startribune.com/elusive-bear-survi...
4,2025-08-06-02-05-33 +0000,bbc,Hiroshima marks 80 years since atomic bombing,https://www.bbc.com/news/articles/cm2v58qrjq0o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2297/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
54,trump,49
427,out,12
89,war,12
251,new,11
57,tariffs,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
9,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...,91
88,2025-08-05-22-21-43 +0000,nypost,"Trump rips ‘no talent’ Gayle King, citing New ...",https://nypost.com/2025/08/05/media/trump-rips...,86
145,2025-08-05-19-42-59 +0000,nypost,Zelensky says Ukraine ready to ink drone deal ...,https://nypost.com/2025/08/05/us-news/zelensky...,86
146,2025-08-05-19-37-53 +0000,nyt,"Ahead of Trump’s Trade Deadline, Firms Rushed ...",https://www.nytimes.com/2025/08/01/business/ta...,85
23,2025-08-06-01-00-00 +0000,wsj,Federal Reserve governor Adriana Kugler is set...,https://www.wsj.com/economy/central-banking/tr...,84


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
9,91,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...
251,53,2025-08-05-13-39-25 +0000,nypost,CEO ignored ‘red flags’ ahead of ‘preventable’...,https://nypost.com/2025/08/05/world-news/titan...
34,43,2025-08-06-00-26-29 +0000,nyt,California Democrats Look to Redraw House Map ...,https://www.nytimes.com/2025/08/05/us/politics...
213,42,2025-08-05-16-34-00 +0000,wsj,The Swiss president is heading to the U.S. in ...,https://www.wsj.com/economy/trade/swiss-presid...
73,40,2025-08-05-22-53-49 +0000,nypost,Netanyahu believes occupying Gaza is the only ...,https://nypost.com/2025/08/05/world-news/netan...
205,37,2025-08-05-16-53-34 +0000,nypost,Former X CEO Linda Yaccarino lands new gig at ...,https://nypost.com/2025/08/05/business/former-...
146,32,2025-08-05-19-37-53 +0000,nyt,"Ahead of Trump’s Trade Deadline, Firms Rushed ...",https://www.nytimes.com/2025/08/01/business/ta...
253,32,2025-08-05-13-27-00 +0000,cbc,"India, Russia hit back after U.S. accuses Modi...",https://www.cbc.ca/news/world/india-us-russia-...
176,32,2025-08-05-18-02-25 +0000,nypost,Tulsi Gabbard tells ‘Pod Force One’ she believ...,https://nypost.com/2025/08/05/us-news/tulsi-ga...
33,31,2025-08-06-00-31-37 +0000,nypost,Ex-Miami Heat security worker stole and sold $...,https://nypost.com/2025/08/05/sports/retried-m...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
